In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout
)
from tensorflow.keras.utils import to_categorical

In [ ]:
sns.set(style="darkgrid")
np.random.seed(42)
tf.random.set_seed(42)

LOAD DATASET

In [ ]:
digits = load_digits()

X = digits.images     
y = digits.target     
print("Original image shape:", X.shape)

VISUALIZE SAMPLE IMAGES

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i], cmap="gray")
    plt.title(f"Digit: {y[i]}")
    plt.axis("off")
plt.suptitle("Sample Digits (8x8 Images)")
plt.show()